The CIFAR-10 dataset consists of 60000 32x32 colour images in 10 classes, with 6000 images per class. There are 50000 training images and 10000 test images.
We are going to build a CNN model to classify these images and then we will use transfer learning to find out which one worked better.
Let's start with out dataset and take look at it.
* Costume CNN
* ResNet50 with fine tuning
* MobileNet with fine tuning
* VGG-19 with fune tuning

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Conv2D, Dropout
from tensorflow.keras.layers import GlobalAveragePooling2D, UpSampling2D, Input
from tensorflow.keras.layers import MaxPooling2D, BatchNormalization
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
import scipy as sp

In [ ]:
(train_data, train_labels),(test_data, test_labels) = tf.keras.datasets.cifar10.load_data()

As we can see in the cell below out data type is unit8 and we need to change change it to float32. Besides nomalizing our data would be helpfull. That would wrap up out preprocessing for images. We'll see more when we are using transfer learning.

In [ ]:
train_data.shape, train_labels.shape, train_data.dtype, test_data.dtype

Before changing our data let's see some examples of it to have better sight of our dataset.

In [ ]:
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer',
               'dog', 'frog', 'horse', 'ship', 'truck']
plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_data[i])
    # The CIFAR labels happen to be arrays,which is why you need the extra index
    plt.xlabel(class_names[train_labels[i][0]])
plt.show()

In [ ]:
train_data = train_data.astype(np.float32)
test_data = test_data.astype(np.float32)
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)
train_data /= 255.
test_data /= 255.

In [ ]:
print(train_data.shape)
print(test_data.shape)

In [ ]:
inp = Input(shape=(32,32,3))
x = Conv2D(filters=32, kernel_size=(3,3),activation='relu', 
           padding='same')(inp)
x = Conv2D(filters=32, kernel_size=(3,3),activation='relu', 
           padding='same')(x)
x = MaxPooling2D(pool_size=2, strides=2,padding='valid')(x)
x = Conv2D(filters=64, kernel_size=(3,3),activation='relu', 
           padding='same')(x)
x = Conv2D(filters=64, kernel_size=(3,3),activation='relu', 
           padding='same')(x)
x = MaxPooling2D(pool_size=2, strides=2, padding='valid')(x)
x = Flatten()(x)
x = Dropout(0.4)(x)
x = Dense(units=64, activation='relu')(x)
x = Dense(units=10, activation='softmax')(x)
model_costume_cnn = Model(inp, x)
model_costume_cnn.summary()

In [ ]:
model_costume_cnn.compile(optimizer='adam', 
                          loss='categorical_crossentropy', 
                          metrics=['accuracy'])
history_costume_cnn = model_costume_cnn.fit(train_data,train_labels, 
                batch_size=8, epochs=30, validation_split=0.15)

In [ ]:
f,ax=plt.subplots(2,1,figsize=(10,10)) 

#Assigning the first subplot to graph training loss and validation loss
ax[0].plot(history_costume_cnn.history['loss'],label='Training Loss')
ax[0].plot(history_costume_cnn.history['val_loss'],label='Validation Loss')

#Plotting the training accuracy and validation accuracy
ax[1].plot(history_costume_cnn.history['accuracy'],label='Training Accuracy')
ax[1].plot(history_costume_cnn.history['val_accuracy'],label='Validation Accuracy')

plt.legend()

As we can see in the plots, our model is overfitted at epoch=10. we can use earlystop in order to prevent this issue but let's use some pretrained model and compare our model to them.

# ResNet50 with fine tuning
we will use resnet50 model without its top and we will freeze the layers wieghts. After training and fitting data our weights would be something reasonable, then we will unfreeze all the weight and fit the data again.

In [ ]:
(train_data, train_labels),(test_data, test_labels) = tf.keras.
datasets.cifar10.load_data()
train_data = tf.keras.applications.resnet50.preprocess_input(train_data)
test_data = tf.keras.applications.resnet50.preprocess_input(test_data)

In [ ]:
resnet = tf.keras.applications.ResNet50(input_shape=(224,224,3), 
                include_top=False, weights='imagenet', classes=10)
resnet.trainable = False
inputs = Input((32,32,3))
x = UpSampling2D((7,7))(inputs)
x = resnet(x)
x = GlobalAveragePooling2D()(x)
x = Dropout(0.3)(x)
x = Dense(units=128, activation='relu')(x)
x = BatchNormalization()(x)
output = Dense(units=10,activation = 'softmax')(x)
model_resnet = Model(inputs, output)
model_resnet.summary()

In [ ]:
resnet_learning_rate = 0.0001
model_resnet.compile(optimizer=Adam(learning_rate=resnet_learning_rate)
        ,loss='sparse_categorical_crossentropy',metrics=['accuracy'])
savebest = tf.keras.callbacks.ModelCheckpoint(
    'resnet_beforefinetuning.h5', save_best_only=True);
history_resnet = model_resnet.fit(train_data, train_labels, 
    batch_size=64, epochs=15, validation_split=0.15, 
                                  callbacks=[savebest])

In [ ]:
model_resnet = tf.keras.models.load_model('resnet_beforefinetuning.h5')
model_resnet.trainable = True
model_resnet.compile(optimizer=RMSprop(
    learning_rate=resnet_learning_rate/10),
        loss='sparse_categorical_crossentropy',metrics=['accuracy'])
history_resnet = model_resnet.fit(train_data, train_labels, 
                    batch_size=32, epochs=3, validation_split=0.15)

In [ ]:
f,ax=plt.subplots(2,1,figsize=(10,10)) 
#Assigning the first subplot to graph training loss and validation loss
ax[0].plot(history_resnet.history['loss'],label='Training Loss')
ax[0].plot(history_resnet.history['val_loss'],label='Validation Loss')
#Plotting the training accuracy and validation accuracy
ax[1].plot(history_resnet.history['accuracy'],label='Training Accuracy')
ax[1].plot(history_resnet.history['val_accuracy'],label='Validation Accuracy')

plt.legend()

In [ ]:
model_resnet.evaluate(test_data,test_labels)

# MobileNetV2 

In [ ]:
(train_data, train_labels),(test_data, test_labels) = tf.keras.datasets.cifar10.load_data()
train_data = tf.keras.applications.mobilenet_v3.preprocess_input(train_data)
test_data = tf.keras.applications.mobilenet_v3.preprocess_input(test_data)
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

In [ ]:
mobilenet = tf.keras.applications.MobileNetV3Small(input_shape=(96,96,3), include_top=False, weights='imagenet', classes=10)
mobilenet.trainable = False

inputs = Input((32,32,3))

x = UpSampling2D((3,3))(inputs)

x = mobilenet(x)

x = GlobalAveragePooling2D()(x)
x = BatchNormalization()(x)
x = Dropout(0.4)(x)

x = Dense(units=128, activation='relu')(x)
x = BatchNormalization()(x)


output = Dense(units=10,activation = 'softmax')(x)
model_mobilenet = Model(inputs, output)
model_mobilenet.summary()

In [ ]:
model_mobilenet.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
savebest = tf.keras.callbacks.ModelCheckpoint('mobile_beforefinetuning.h5', save_best_only=True)
history_mobilenet_1 = model_mobilenet.fit(train_data, train_labels, batch_size=64, epochs=5, validation_split=0.15, callbacks=[savebest])

In [ ]:
model_mobilenet = tf.keras.models.load_model('mobile_beforefinetuning.h5')
model_mobilenet.trainable = True
model_mobilenet.compile(optimizer=RMSprop(0.00001),loss='categorical_crossentropy',metrics=['accuracy'])
history_mobilenet = model_mobilenet.fit(train_data, train_labels, batch_size=32, epochs=20, validation_split=0.15)

In [ ]:
f,ax=plt.subplots(2,1,figsize=(10,10)) 

#Assigning the first subplot to graph training loss and validation loss
ax[0].plot(history_mobilenet.history['loss'],label='Training Loss')
ax[0].plot(history_mobilenet.history['val_loss'],label='Validation Loss')

#Plotting the training accuracy and validation accuracy
ax[1].plot(history_mobilenet.history['accuracy'],label='Training Accuracy')
ax[1].plot(history_mobilenet.history['val_accuracy'],label='Validation Accuracy')

plt.legend()

So MobileNetV3 with fine tuning did actually a pretty good job. According to the plot the model is obviously not overfitting and we have solid accuracy of 87%. Let's see if VGG-19 can do a better job!

# VGG-19

In [ ]:
(train_data, train_labels),(test_data, test_labels) = tf.keras.datasets.cifar10.load_data()
train_data = tf.keras.applications.vgg19.preprocess_input(train_data)
test_data = tf.keras.applications.vgg19.preprocess_input(test_data)
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

In [ ]:
vgg19 = tf.keras.applications.VGG19(input_shape=(224,224,3), include_top=False, weights='imagenet', classes=10)
vgg19.trainable = False

inputs = Input((32,32,3))

x = UpSampling2D((7,7))(inputs)

x = vgg19(x)

x = GlobalAveragePooling2D()(x)
x = BatchNormalization()(x)
x = Dropout(0.3)(x)

x = Dense(units=128, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.3)(x)

output = Dense(units=10,activation = 'softmax')(x)
model_vgg19 = Model(inputs, output)
model_vgg19.summary()

In [ ]:
initial_epochs = 17
model_vgg19.compile(optimizer=Adam(0.001),loss='categorical_crossentropy',metrics=['accuracy'])
savebest = tf.keras.callbacks.ModelCheckpoint('vgg19_beforefinetuning.h5', save_best_only=True)
history_vgg19_1 = model_vgg19.fit(train_data, train_labels, batch_size=64, epochs=initial_epochs, validation_split=0.15, callbacks=[savebest])

In [ ]:
model_vgg19 = tf.keras.models.load_model('vgg19_beforefinetuning.h5')
model_vgg19.trainable = True
model_vgg19.compile(optimizer=RMSprop(0.00001),loss='categorical_crossentropy',metrics=['accuracy'])
history_vgg19 = model_vgg19.fit(train_data, train_labels, batch_size=32, epochs=5, validation_split=0.15)

In [ ]:
f,ax=plt.subplots(2,1,figsize=(10,10)) 

#Assigning the first subplot to graph training loss and validation loss
ax[0].plot(history_vgg19.history['loss'],label='Training Loss')
ax[0].plot(history_vgg19.history['val_loss'],label='Validation Loss')

#Plotting the training accuracy and validation accuracy
ax[1].plot(history_vgg19.history['accuracy'],label='Training Accuracy')
ax[1].plot(history_vgg19.history['val_accuracy'],label='Validation Accuracy')

plt.legend()